# Basic Time Series Forecasting

This notebook covers fundamental forecasting techniques.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.exponential_smoothing.ets import ETSModel
from statsmodels.tsa.holtwinters import ExponentialSmoothing
from sklearn.metrics import mean_absolute_error, mean_squared_error
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

## 1. Load Data

In [ ]:
# Load your preprocessed data
# df = pd.read_csv('../data/processed/clean_data.csv', parse_dates=['timestamp'], index_col='timestamp')

## 2. Train/Test Split

In [ ]:
def train_test_split_ts(data, test_size=0.2):
    split_point = int(len(data) * (1 - test_size))
    train = data[:split_point]
    test = data[split_point:]
    return train, test

# train_data, test_data = train_test_split_ts(df['value'])

## 3. Naive Forecast (Baseline)

In [ ]:
def naive_forecast(train_data, steps):
    return [train_data.iloc[-1]] * steps

def seasonal_naive_forecast(train_data, steps, season_length):
    seasonal_values = train_data.iloc[-season_length:]
    forecast = []
    for i in range(steps):
        forecast.append(seasonal_values.iloc[i % season_length])
    return forecast

## 4. ARIMA Model

In [ ]:
def fit_arima(train_data, order=(1, 1, 1)):
    model = ARIMA(train_data, order=order)
    fitted_model = model.fit()
    return fitted_model

def forecast_arima(fitted_model, steps):
    forecast = fitted_model.forecast(steps=steps)
    return forecast

## 5. Exponential Smoothing

In [ ]:
def fit_exp_smoothing(train_data, seasonal_periods=24):
    model = ExponentialSmoothing(train_data, 
                                 trend='add', 
                                 seasonal='add', 
                                 seasonal_periods=seasonal_periods)
    fitted_model = model.fit()
    return fitted_model

## 6. Model Evaluation

In [ ]:
def evaluate_forecast(actual, predicted):
    mae = mean_absolute_error(actual, predicted)
    mse = mean_squared_error(actual, predicted)
    rmse = np.sqrt(mse)
    mape = np.mean(np.abs((actual - predicted) / actual)) * 100
    
    return {
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'MAPE': mape
    }

## 7. Visualization

In [ ]:
def plot_forecast_comparison(train_data, test_data, forecasts, model_names):
    plt.figure(figsize=(15, 8))
    
    # Plot training data
    plt.plot(train_data.index, train_data.values, label='Training Data', color='blue')
    
    # Plot test data
    plt.plot(test_data.index, test_data.values, label='Actual', color='black', linewidth=2)
    
    # Plot forecasts
    colors = ['red', 'green', 'orange', 'purple']
    for i, (forecast, name) in enumerate(zip(forecasts, model_names)):
        plt.plot(test_data.index, forecast, label=f'{name} Forecast', 
                color=colors[i % len(colors)], linestyle='--')
    
    plt.legend()
    plt.title('Forecast Comparison')
    plt.xlabel('Time')
    plt.ylabel('Value')
    plt.show()